<center><p float="center">
  <img src="https://upload.wikimedia.org/wikipedia/commons/e/e9/4_RGB_McCombs_School_Brand_Branded.png" width="300" height="100"/>
  <img src="https://mma.prnewswire.com/media/1458111/Great_Learning_Logo.jpg?p=facebook" width="200" height="100"/>
</p></center>

<h1><center><font size=10>Artificial Intelligence and Machine Learning</center></font></h1>
<h1><center>LLMs and Prompt Engineering</center></h1>

<center><p float="center">
  <img src="https://images.pexels.com/photos/4185957/pexels-photo-4185957.jpeg?auto=compress&cs=tinysrgb&w=1260&h=750&dpr=1" width="720"/>
</p></center>

<center><font size=6>News Article Categorization and Summarization</font></center>

## Problem Statement

### Business Context

In the dynamic landscape of the media and news industry, the ability to swiftly categorize and curate content has become a strategic imperative. The vast volume of information demands efficient systems to organize and present content to the audience.

The media industry, being the pulse of information dissemination, grapples with the continuous influx of news articles spanning diverse topics. Ensuring that the right articles reach the right audience promptly is not just a logistical necessity but a critical component in retaining and engaging audiences in an age of information overload.

Common Industry Challenges:
Amidst the ceaseless flow of news, organizations encounter challenges such as:
- Information Overload: The sheer volume of news articles makes manual categorization impractical.
- Timeliness: Delays in categorizing news articles can result in outdated or misplaced content.

### Problem Definition

E-news Express, a news aggregation startup, faces the challenge of categorizing the news articles collected. With news articles covering sports, entertainment, politics, and more, the need for an advanced and automated system to categorize them has become increasingly evident. The manual efforts required for categorizing such a diverse range of news articles are substantial, and human errors in the categorization of news articles can lead to reputational damage for the startup. There is also the factor of delays and potential inaccuracies. To streamline and optimize this process, the organization recognizes the imperative of adopting cutting-edge technologies, particularly machine learning, to automate and enhance the categorization of content.

As a data scientist on the E-news Express data team, the task is to analyze the text in news articles and build an unsupervised learning model for categorizing them. The categorization will be followed up by a summarization of the news article so that users can read through the summary first and then click on a relevant link to dive in deeper if needed. The goal is to optimize the categorization and summarization process, ensuring timely and personalized delivery.

### Data Dictionary

- **Article**: The main body of the news article

## Installing and Importing Necessary Libraries

In [4]:
# Installation for Google Colab with GPU (CUDA) support
!pip install llama-cpp-python --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu121 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 551.3/551.3 MB 2.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.5 MB/s eta 0:00:00


In [5]:
# For downloading the models from HF Hub
!pip install huggingface_hub datasets -q

In [6]:
# Importing library for data manipulation
import pandas as pd

# Function to download the model from the Hugging Face model hub
from huggingface_hub import hf_hub_download

# Importing the Llama class from the llama_cpp module
from llama_cpp import Llama

# Importing the json library
import json

#Importing the datasets library from huggingface
from datasets import load_dataset

## Load the dataset

In [15]:
# Option 1: Download from a public URL (GitHub, S3, etc.)
# Replace the URL below with your file's public URL
!wget -q "https://raw.githubusercontent.com/visubramaniam/AI-RAG-GENAI/main/data/articles.csv" -O articles.csv

# Option 2: Download from Google Drive using gdown (if file is shared publicly)
# First, get the shareable link from Google Drive, extract the file ID
# Example link: https://drive.google.com/file/d/FILE_ID/view?usp=sharing
# !pip install gdown -q
# !gdown "FILE_ID" -O articles.csv

# Option 3: Download from AWS S3 (public bucket)
# !wget -q "https://YOUR_BUCKET.s3.amazonaws.com/articles.csv" -O articles.csv

In [16]:
# Load the uploaded CSV file
data = pd.read_csv("articles.csv")

## Data Overview

In [17]:
#Printing the first few rows.
data.head()

,Article
0,A New Push to Loosen New York's Divorce Law La...
1,Memos Warned of Billing Fraud by Firm in Iraq ...
2,"New Chechen Leader Vows Peace, Poll Criticized..."
3,WWW KOTV.com _ Two of three UN workers kidnapp...
4,U.S. State Dept Finishes Review of Iraq Aid Pl...


In [18]:
#Number of news articles.
print(len(data))

100


## Model Building

### Loading the model

In [20]:
from dotenv import load_dotenv
import os
HF_TOKEN = os.getenv('HUGGINGFACE_TOKEN')

In [21]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGUF"
model_basename = "llama-2-13b-chat.Q5_K_M.gguf" # the model is in gguf format

In [24]:
# Using hf_hub_download to download a model from the Hugging Face model hub
# The repo_id parameter specifies the model name or path in the Hugging Face repository
# The filename parameter specifies the name of the file to download
model_path = hf_hub_download(
    repo_id="TheBloke/Llama-2-13B-chat-GGUF",
    filename=model_basename,
    token=HF_TOKEN
)

llama-2-13b-chat.Q5_K_M.gguf:   0%|          | 0.00/9.23G [00:00<?, ?B/s]

In [25]:
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2,  # CPU cores
    n_batch=512,  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=43,  # Change this value based on your model and your GPU VRAM pool.
    n_ctx=4096,  # Context window
)

ggml_cuda_init: GGML_CUDA_FORCE_MMQ:    yes
ggml_cuda_init: GGML_CUDA_FORCE_CUBLAS: no
ggml_cuda_init: found 1 CUDA devices:
  Device 0: Tesla T4, compute capability 7.5, VMM: yes
llama_model_load_from_file_impl: using device CUDA0 (Tesla T4) - 14992 MiB free
llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGUF/snapshots/4458acc949de0a9914c3eab623904d4fe999050a/llama-2-13b-chat.Q5_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120


### Defining Model Response Parameters

In [26]:
def generate_llama_response(instruction, review):

    # System message explicitly instructing not to include the review text
    system_message = """
        [INST]<<SYS>>
        {}
        <</SYS>>[/INST]
    """.format(instruction)

    # Combine user_prompt and system_message to create the prompt
    prompt = f"{review}\n{system_message}"

    # Generate a response from the LLaMA model
    response = lcpp_llm(
        prompt=prompt,
        max_tokens=1024,
        temperature=0,
        top_p=0.95,
        repeat_penalty=1.2,
        top_k=50,
        stop=['INST'],
        echo=False,
        seed=42,
    )

    # Extract the sentiment from the response
    response_text = response["choices"][0]["text"]
    return response_text

- **`max_tokens`**: This parameter **specifies the maximum number of tokens that the model should generate** in response to the prompt.

- **`temperature`**: This parameter **controls the randomness of the generated response**. A higher temperature value will result in a more random response, while a lower temperature value will result in a more predictable response.

- **`top_p`**: This parameter **controls the diversity of the generated response by establishing a cumulative probability cutoff for token selection**. A higher value of top_p will result in a more diverse response, while a lower value will result in a less diverse response.

- **`repeat_penalty`**: This parameter **controls the penalty for repeating tokens in the generated response**. A higher value of repeat_penalty will result in a lower probability of repeating tokens, while a lower value will result in a higher probability of repeating tokens.

- **`top_k`**: This parameter **controls the maximum number of most-likely next tokens to consider** when generating the response at each step.

- **`stop`**: This parameter is a **list of tokens that are used to dynamically stop response generation** whenever the tokens in the list are encountered.

- **`echo`**: This parameter **controls whether the input (prompt) to the model should be returned** in the model response.

- **`seed`**: This parameter **specifies a seed value that helps replicate results**.


## Classifying the news articles

In [27]:
data_1 = data.copy()

In [29]:
# defining the instructions for the model
instruction_1 = """
    You are an AI analyzing news articles. Tag the given article using one or more of the below mentioned categories only depending upon the content of the article:
    - World
    - Sports
    - Business
    - Sci/Tech
"""

In [30]:
data_1['model_response_1'] = data_1['Article'].apply(lambda x: generate_llama_response(instruction_1, x))

llama_perf_context_print:        load time =     669.03 ms
llama_perf_context_print: prompt eval time =     668.57 ms /   129 tokens (    5.18 ms per token,   192.95 tokens per second)
llama_perf_context_print:        eval time =    1252.28 ms /    21 runs   (   59.63 ms per token,    16.77 tokens per second)
llama_perf_context_print:       total time =    1937.91 ms /   150 tokens
llama_perf_context_print:    graphs reused =         20
Llama.generate: 1 prefix-match hit, remaining 116 prompt tokens to eval
llama_perf_context_print:        load time =     669.03 ms
llama_perf_context_print: prompt eval time =     216.70 ms /   116 tokens (    1.87 ms per token,   535.30 tokens per second)
llama_perf_context_print:        eval time =    8411.17 ms /   137 runs   (   61.40 ms per token,    16.29 tokens per second)
llama_perf_context_print:       total time =    8755.94 ms /   253 tokens
llama_perf_context_print:    graphs reused =        132
Llama.generate: 1 prefix-match hit, remaining 

In [31]:
#Printing the model's response.
data_1["model_response_1"]

,model_response_1
0,"Sure! Here's the article you provided, tagged..."
1,"Sure, I can help you with that! Here's the ar..."
2,"Sure! Here's the article you provided, tagged..."
3,"Sure, I can help you with that! Based on the ..."
4,"Sure, I can help you with that! Here's the ar..."
...,...
95,"Sure! Here's the article you provided, tagged..."
96,"Sure, I can help you with that! Here's the ar..."
97,"Sure! Here's the article you provided, tagged..."
98,"Sure, I can help you with that! Here's the ar..."


In [32]:
#Printing the model's response.
data_1["model_response_1"][0]

" Sure! Here's the article you provided, tagged according to its content:\n\n        ["

In [33]:
def extract_label(model_response):
    if 'business' in model_response.lower():
        return 'Business'
    elif 'world' in model_response.lower():
        return 'World'
    elif 'sports' in model_response.lower():
        return 'Sports'
    else:
      return 'Sci/Tech'

In [34]:
data_1["Label"] = data_1["model_response_1"].apply(extract_label)

In [35]:
final_data_1 = data_1.drop(['model_response_1'], axis=1)
final_data_1.head()

,Article,Label
0,A New Push to Loosen New York's Divorce Law La...,Sci/Tech
1,Memos Warned of Billing Fraud by Firm in Iraq ...,Business
2,"New Chechen Leader Vows Peace, Poll Criticized...",World
3,WWW KOTV.com _ Two of three UN workers kidnapp...,Business
4,U.S. State Dept Finishes Review of Iraq Aid Pl...,World


## Classifying the news article and returning a structured output

In [36]:
data_2 = data.copy()

In [38]:
# defining the instructions for the model
instruction_2 = """
    You are an AI analyzing news articles. Tag the given article using one or more of the below mentioned categories only and not any other depending upon the content of the article:
    - World
    - Sports
    - Business
    - Sci/Tech

    Format the output as a JSON object with a single key-value pair as shown below:
    {"label": "your_label_prediction"}
"""

In [39]:
data_2['model_response_2'] = data_2['Article'].apply(lambda x: generate_llama_response(instruction_2, x))

Llama.generate: 1 prefix-match hit, remaining 166 prompt tokens to eval
llama_perf_context_print:        load time =     669.03 ms
llama_perf_context_print: prompt eval time =     449.80 ms /   166 tokens (    2.71 ms per token,   369.05 tokens per second)
llama_perf_context_print:        eval time =    2596.83 ms /    34 runs   (   76.38 ms per token,    13.09 tokens per second)
llama_perf_context_print:       total time =    3077.34 ms /   200 tokens
llama_perf_context_print:    graphs reused =         32
Llama.generate: 1 prefix-match hit, remaining 154 prompt tokens to eval
llama_perf_context_print:        load time =     669.03 ms
llama_perf_context_print: prompt eval time =     365.25 ms /   154 tokens (    2.37 ms per token,   421.63 tokens per second)
llama_perf_context_print:        eval time =    2056.62 ms /    26 runs   (   79.10 ms per token,    12.64 tokens per second)
llama_perf_context_print:       total time =    2445.67 ms /   180 tokens
llama_perf_context_print:    g

In [41]:
#Printing the model's response.
data_2["model_response_2"][0]

' Sure, I can help you with that! Based on the content of the article provided, I would tag it as follows:\n\n{"label": "Business"}'

In [64]:
# defining a function to parse the JSON output from the model
def extract_json_data(json_str):
    try:
        # Find the indices of the opening and closing curly braces
        json_start = json_str.find('{')
        json_end = json_str.rfind('}')

        if json_start != -1 and json_end != -1:
            extracted_json = json_str[json_start:json_end + 1]  # Extract the JSON object
            
            # Fix common escape issues in LLM-generated JSON
            # Replace invalid backslash escapes (but preserve valid ones like \n, \t, \\, \")
            import re
            # Fix invalid escape sequences by escaping lone backslashes
            extracted_json = re.sub(r'\\(?!["\\/bfnrtu])', r'\\\\', extracted_json)
            
            data_dict = json.loads(extracted_json)
            return data_dict
        else:
            print(f"Warning: JSON object not found in response: {json_str}")
            return {}
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")
        return {}

In [65]:
#Applying the function.
data_2["model_response_2_parsed"] = data_2["model_response_2"].apply(extract_json_data)

In [66]:
#Printing the labels predicted by the model.
data_2["model_response_2_parsed"]

,model_response_2_parsed
0,{'label': 'Business'}
1,{'label': 'Business'}
2,{'label': 'World'}
3,{'label': 'World'}
4,{'label': 'World'}
...,...
95,{'label': 'Sci/Tech'}
96,{'label': 'Sci/Tech'}
97,{'label': 'Sci/Tech'}
98,{'label': 'Business'}


In [67]:
data_2[data_2["model_response_2_parsed"]=={}]

,Article,model_response_2,model_response_2_parsed


In [68]:
model_response_parsed_df_2 = pd.json_normalize(data_2['model_response_2_parsed'])
model_response_parsed_df_2.head()

,label
0,Business
1,Business
2,World
3,World
4,World


In [47]:
data_with_parsed_model_output_2 = pd.concat([data_2, model_response_parsed_df_2], axis=1)
data_with_parsed_model_output_2.head()

,Article,model_response_2,model_response_2_parsed,label
0,A New Push to Loosen New York's Divorce Law La...,"Sure, I can help you with that! Based on the ...",{'label': 'Business'},Business
1,Memos Warned of Billing Fraud by Firm in Iraq ...,"Sure! Here's the article you provided, tagged...",{'label': 'Business'},Business
2,"New Chechen Leader Vows Peace, Poll Criticized...","Sure, I can help you with that! Based on the ...",{'label': 'World'},World
3,WWW KOTV.com _ Two of three UN workers kidnapp...,"Sure, I can help you with that! Based on the ...",{'label': 'World'},World
4,U.S. State Dept Finishes Review of Iraq Aid Pl...,"Sure, I can help you with that! Based on the ...",{'label': 'World'},World


In [48]:
final_data_2 = data_with_parsed_model_output_2.drop(['model_response_2','model_response_2_parsed'], axis=1)
final_data_2.head()

,Article,label
0,A New Push to Loosen New York's Divorce Law La...,Business
1,Memos Warned of Billing Fraud by Firm in Iraq ...,Business
2,"New Chechen Leader Vows Peace, Poll Criticized...",World
3,WWW KOTV.com _ Two of three UN workers kidnapp...,World
4,U.S. State Dept Finishes Review of Iraq Aid Pl...,World


## Classifying the news articles and generating a headline

In [49]:
data_3 = data.copy()

In [50]:
# defining the instructions for the model
instruction_3 = """
    You are an AI analyzing news articles. Tag the given article using one or more of the below mentioned categories only depending upon the content of the article:
    - World
    - Sports
    - Business
    - Sci/Tech

    Once done,craft a compelling headline summarizing the key insights and significance of the article.Ensure the headline is attention-grabbing and concise while accurately reflecting the content and output it.

    Provide the output in a JSON format with the following keys:
    {
        "Label": "your_label_prediction",
        "Headline": "your_headline_prediction",
    }

    Only return the JSON, do not return any other information and remove the extra spaces.
"""

In [51]:
data_3['model_response_3'] = data_3['Article'].apply(lambda x: generate_llama_response(instruction_3, x).replace('\n', ''))

Llama.generate: 1 prefix-match hit, remaining 248 prompt tokens to eval
llama_perf_context_print:        load time =     669.03 ms
llama_perf_context_print: prompt eval time =     526.39 ms /   248 tokens (    2.12 ms per token,   471.13 tokens per second)
llama_perf_context_print:        eval time =    2876.64 ms /    37 runs   (   77.75 ms per token,    12.86 tokens per second)
llama_perf_context_print:       total time =    3433.71 ms /   285 tokens
llama_perf_context_print:    graphs reused =         35
Llama.generate: 1 prefix-match hit, remaining 236 prompt tokens to eval
llama_perf_context_print:        load time =     669.03 ms
llama_perf_context_print: prompt eval time =     474.06 ms /   236 tokens (    2.01 ms per token,   497.83 tokens per second)
llama_perf_context_print:        eval time =    3456.55 ms /    43 runs   (   80.38 ms per token,    12.44 tokens per second)
llama_perf_context_print:       total time =    3966.46 ms /   279 tokens
llama_perf_context_print:    g

In [52]:
data_3["model_response_3_parsed"] = data_3["model_response_3"].apply(extract_json_data)

In [53]:
data_3["model_response_3_parsed"]

,model_response_3_parsed
0,"{'Label': 'Business', 'Headline': 'New York's ..."
1,"{'Label': 'Business', 'Headline': 'Memos Warne..."
2,"{'Label': 'World', 'Headline': 'New Chechen Le..."
3,"{'Label': 'World', 'Headline': 'Two UN Workers..."
4,"{'Label': 'World', 'Headline': 'U.S. State Dep..."
...,...
95,"{'Label': 'Sci/Tech', 'Headline': 'Atomic Visi..."
96,"{'Label': 'Sci/Tech', 'Headline': 'Ninth-grade..."
97,"{'Label': 'Sci/Tech', 'Headline': 'Congress Wa..."
98,"{'Label': 'Business', 'Headline': 'IBM Jumps i..."


In [54]:
data_3[data_3["model_response_3_parsed"]=={}]

,Article,model_response_3,model_response_3_parsed


In [55]:
data_3["model_response_3"][43]

'    {        "Label": "Sports",        "Headline": "Hewitt Wins Washington Open, Fine-Tunes Game for US Open"    }'

In [56]:
data_3["model_response_3_parsed"][43] = {"label":"Sports","Headline":"Hewitt wins Washington Open: Australia's Lleyton Hewitt Captures $US500,000 Tournament"}

In [57]:
model_response_parsed_df_3 = pd.json_normalize(data_3['model_response_3_parsed']).drop(["label"],axis=1)
model_response_parsed_df_3.head()

,Label,Headline
0,Business,New York's Divorce Law Under Scrutiny: A Push ...
1,Business,Memos Warned of Billing Fraud by Firm in Iraq:...
2,World,"New Chechen Leader Vows Peace, Poll Criticized"
3,World,Two UN Workers Kidnapped in Afghanistan Call H...
4,World,U.S. State Dept Finishes Review of Iraq Aid Plan


In [58]:
data_with_parsed_model_output_3 = pd.concat([data_3, model_response_parsed_df_3], axis=1)
data_with_parsed_model_output_3.head()

,Article,model_response_3,model_response_3_parsed,Label,Headline
0,A New Push to Loosen New York's Divorce Law La...,"{""Label"": ""Business"",""Headline"": ""New York's D...","{'Label': 'Business', 'Headline': 'New York's ...",Business,New York's Divorce Law Under Scrutiny: A Push ...
1,Memos Warned of Billing Fraud by Firm in Iraq ...,"{ ""Label"": ""Business"", ""Head...","{'Label': 'Business', 'Headline': 'Memos Warne...",Business,Memos Warned of Billing Fraud by Firm in Iraq:...
2,"New Chechen Leader Vows Peace, Poll Criticized...","{ ""Label"": ""World"", ""Headlin...","{'Label': 'World', 'Headline': 'New Chechen Le...",World,"New Chechen Leader Vows Peace, Poll Criticized"
3,WWW KOTV.com _ Two of three UN workers kidnapp...,"{""Label"": ""World"",""Headline"": ""Two UN Workers ...","{'Label': 'World', 'Headline': 'Two UN Workers...",World,Two UN Workers Kidnapped in Afghanistan Call H...
4,U.S. State Dept Finishes Review of Iraq Aid Pl...,"{ ""Label"": ""World"", ""Headlin...","{'Label': 'World', 'Headline': 'U.S. State Dep...",World,U.S. State Dept Finishes Review of Iraq Aid Plan


In [59]:
final_data_3 = data_with_parsed_model_output_3.drop(['model_response_3','model_response_3_parsed'], axis=1)
final_data_3.head()

,Article,Label,Headline
0,A New Push to Loosen New York's Divorce Law La...,Business,New York's Divorce Law Under Scrutiny: A Push ...
1,Memos Warned of Billing Fraud by Firm in Iraq ...,Business,Memos Warned of Billing Fraud by Firm in Iraq:...
2,"New Chechen Leader Vows Peace, Poll Criticized...",World,"New Chechen Leader Vows Peace, Poll Criticized"
3,WWW KOTV.com _ Two of three UN workers kidnapp...,World,Two UN Workers Kidnapped in Afghanistan Call H...
4,U.S. State Dept Finishes Review of Iraq Aid Pl...,World,U.S. State Dept Finishes Review of Iraq Aid Plan


## Classifying the news articles, generating a headline, and generating a summary

In [60]:
data_4 = data.copy()

In [69]:
# defining the instructions for the model
instruction_4 = """
    You are an AI analyzing news articles. Tag the given article using one or more of the below mentioned categories only depending upon the content of the article:
    - World
    - Sports
    - Business
    - Sci/Tech

    Once done,craft a compelling headline summarizing the key insights and significance of the article.Ensure the headline is attention-grabbing and concise while accurately reflecting the content and output it.

    Further,"Summarize the key points and main arguments presented in the news article, while providing a concise yet comprehensive overview of the information and its implications."
    Provide the output in a JSON format with the following keys:
    {
        "Label": "your_label_prediction",
        "Headline": "your_headline_prediction",
        "Summary": "your_summary_prediction"
    }

    Only return the JSON, do not return any other information and remove the extra spaces.
"""

In [62]:
data_4['model_response_4'] = data_4['Article'].apply(lambda x: generate_llama_response(instruction_4, x).replace('\n', ''))

Llama.generate: 1 prefix-match hit, remaining 298 prompt tokens to eval
llama_perf_context_print:        load time =     669.03 ms
llama_perf_context_print: prompt eval time =     714.48 ms /   298 tokens (    2.40 ms per token,   417.09 tokens per second)
llama_perf_context_print:        eval time =    7161.40 ms /    87 runs   (   82.31 ms per token,    12.15 tokens per second)
llama_perf_context_print:       total time =    7954.54 ms /   385 tokens
llama_perf_context_print:    graphs reused =         83
Llama.generate: 1 prefix-match hit, remaining 286 prompt tokens to eval
llama_perf_context_print:        load time =     669.03 ms
llama_perf_context_print: prompt eval time =     605.22 ms /   286 tokens (    2.12 ms per token,   472.56 tokens per second)
llama_perf_context_print:        eval time =   10450.70 ms /   123 runs   (   84.97 ms per token,    11.77 tokens per second)
llama_perf_context_print:       total time =   11173.07 ms /   409 tokens
llama_perf_context_print:    g

In [70]:
data_4["model_response_4_parsed"] = data_4["model_response_4"].apply(extract_json_data)

In [71]:
data_4["model_response_4_parsed"]

,model_response_4_parsed
0,"{'Label': 'Business', 'Headline': 'New York's ..."
1,"{'Label': 'Business', 'Headline': 'Memos Warne..."
2,"{'Label': 'World', 'Headline': 'New Chechen Le..."
3,"{'Label': 'World', 'Headline': 'Two UN Workers..."
4,"{'Label': 'World', 'Headline': 'U.S. State Dep..."
...,...
95,"{'Label': 'Sci/Tech', 'Headline': 'World's Sha..."
96,"{'Label': 'Sci/Tech', 'Headline': 'Ninth-Grade..."
97,"{'Label': 'World', 'Headline': 'Congress Wants..."
98,"{'Label': 'Business', 'Headline': 'IBM Jumps i..."


In [72]:
data_4[data_4["model_response_4_parsed"]=={}]

,Article,model_response_4,model_response_4_parsed


In [73]:
data_4["model_response_4"][43]

'    {        "Label": "Sports",        "Headline": "Hewitt Wins Washington Open, Fine-Tunes Game for US Open",        "Summary": "Lleyton Hewitt wins the \\$US500,000 (\\$A692,377) Washington Open, defeating Gilles Muller in a convincing 6-3 6-4 victory. The win fine-tunes Hewitt\'s game ahead of the US Open."    }'

In [74]:
data_4["model_response_4_parsed"][43] = {"Label": "Sports","Headline": "Hewitt Wins Washington Open, Fine-Tunes Game for US Open","Summary": "Lleyton Hewitt won the $500,000 Washington Open, defeating Gilles Muller in straight sets.The victory fine-tuned his game ahead of the US Open."}

In [75]:
model_response_parsed_df_4 = pd.json_normalize(data_4['model_response_4_parsed'])
model_response_parsed_df_4.head()

,Label,Headline,Summary
0,Business,New York's Divorce Law Under Scrutiny: A Push ...,Lawyers and judges are advocating for a change...
1,Business,"Memos Warned of Billing Fraud by Firm in Iraq,...",The article reveals memos that suggest Custer ...
2,World,"New Chechen Leader Vows Peace, Poll Criticized",Chechnya's new leader vowed to rebuild the reg...
3,World,Two UN Workers Kidnapped in Afghanistan Call H...,Two of three United Nations workers kidnapped ...
4,World,U.S. State Dept Finishes Review of Iraq Aid Pl...,The U.S. State Department has finished an inte...


In [76]:
data_with_parsed_model_output_4 = pd.concat([data_4, model_response_parsed_df_4], axis=1)
data_with_parsed_model_output_4.head()

,Article,model_response_4,model_response_4_parsed,Label,Headline,Summary
0,A New Push to Loosen New York's Divorce Law La...,"{ ""Label"": ""Business"", ""Head...","{'Label': 'Business', 'Headline': 'New York's ...",Business,New York's Divorce Law Under Scrutiny: A Push ...,Lawyers and judges are advocating for a change...
1,Memos Warned of Billing Fraud by Firm in Iraq ...,"{ ""Label"": ""Business"", ""Head...","{'Label': 'Business', 'Headline': 'Memos Warne...",Business,"Memos Warned of Billing Fraud by Firm in Iraq,...",The article reveals memos that suggest Custer ...
2,"New Chechen Leader Vows Peace, Poll Criticized...","{ ""Label"": ""World"", ""Headlin...","{'Label': 'World', 'Headline': 'New Chechen Le...",World,"New Chechen Leader Vows Peace, Poll Criticized",Chechnya's new leader vowed to rebuild the reg...
3,WWW KOTV.com _ Two of three UN workers kidnapp...,"{""Label"": ""World"",""Headline"": ""Two UN Workers ...","{'Label': 'World', 'Headline': 'Two UN Workers...",World,Two UN Workers Kidnapped in Afghanistan Call H...,Two of three United Nations workers kidnapped ...
4,U.S. State Dept Finishes Review of Iraq Aid Pl...,"{ ""Label"": ""World"", ""Headlin...","{'Label': 'World', 'Headline': 'U.S. State Dep...",World,U.S. State Dept Finishes Review of Iraq Aid Pl...,The U.S. State Department has finished an inte...


In [77]:
final_data_4 = data_with_parsed_model_output_4.drop(['model_response_4','model_response_4_parsed'], axis=1)
final_data_4.head()

,Article,Label,Headline,Summary
0,A New Push to Loosen New York's Divorce Law La...,Business,New York's Divorce Law Under Scrutiny: A Push ...,Lawyers and judges are advocating for a change...
1,Memos Warned of Billing Fraud by Firm in Iraq ...,Business,"Memos Warned of Billing Fraud by Firm in Iraq,...",The article reveals memos that suggest Custer ...
2,"New Chechen Leader Vows Peace, Poll Criticized...",World,"New Chechen Leader Vows Peace, Poll Criticized",Chechnya's new leader vowed to rebuild the reg...
3,WWW KOTV.com _ Two of three UN workers kidnapp...,World,Two UN Workers Kidnapped in Afghanistan Call H...,Two of three United Nations workers kidnapped ...
4,U.S. State Dept Finishes Review of Iraq Aid Pl...,World,U.S. State Dept Finishes Review of Iraq Aid Pl...,The U.S. State Department has finished an inte...


## Conclusion

- We used an LLM to do multiple tasks, one stage at a time
    1. We first identified the category of the news article.
    2. Next, in addition to identifying the category, we generated a headline for it.
    3. Finally, in addition to identifying the category and generating a headline, we also generated a summary for the article.

- To try and improve the model output, one can try the following:
    1. Update the prompt
    2. Update the model parameters (`temparature`, `top_p`, ...)



<font size=6 color='blue'>Power Ahead</font>
___